In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re

In [2]:
data = pd.read_excel(r'C:\Users\cxy\OneDrive\桌面\回国\实习\7-29\小规模测试.xlsx')
df = data.reset_index().drop(columns='index')
df['医院名称'] = df['医院名称'].astype('string')
df

,医院名称
0,伊春市带岭区人民医院
1,大庆市中西医结合医院
2,大同区林源医院
3,杜尔伯特蒙古族自治县中医医院
4,杜尔伯特蒙古族自治县人民医院
5,杜蒙县妇幼保健医院
6,杜尔伯特蒙古族自治县妇幼保健计划生育服务中心
7,大庆市萨尔图区陈立军内科诊所
8,大庆油田四厂医院
9,大庆油田总医院集团南区医院


In [4]:
def resolve(url, headers_list):
    for i in range(5):  # 循环去请求网站
        headers = np.random.choice(headers_list)
        response = requests.get(url, headers=headers, timeout=20)
        if response.status_code == 200:
            break
    return response

In [5]:
# 因为每个界面必定有p_tag以及b_tag, 当爬虫出错时换headers重新爬取界面信息
def find_b_tag(url, headers, p_tag):
    try:
        b_tag = p_tag.find("b")
        return b_tag
    except:
        response = resolve(url, headers_list)
        content = response.text
        soup = BeautifulSoup(content, 'html.parser')
        p_tag = soup.find("p", class_="result")
        if p_tag is None:
            return find_b_tag(url, headers, p_tag)
        else:
            return p_tag.find("b")

In [6]:
headers_list = [
       {
        'user-agent': "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36"
    }, {
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36 Edg/116.0.1938.76'
    }, {
        'user-agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:109.0) Gecko/20100101 Firefox/115.0'
    }, {
        'user-agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:58.0) Gecko/20100101 Firefox/58.0'
    }, {
        'user-agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:88.0) Gecko/20100101 Firefox/88.0'
    }, {
        'user-agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.87 Safari/537.36 SE 2.X MetaSr 1.0'
    }, {
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36'
    }, {
        'user-agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.54 Safari/537.36'
    }, {
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.71 Safari/537.36 Core/1.94.202.400 QQBrowser/11.9.5355.400'
    }, {
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36'
    }, {
        'user-agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Mobile Safari/537.36'
    }
]
headers = np.random.choice(headers_list)

In [7]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

base_url = 'https://www.dayi.org.cn/search?keyword='  # 修改了URL的格式

yiyuan = []
hospital_link = []

for i in range(len(df["医院名称"])): 
    name = df["医院名称"][i]
    # 过滤名称中包含特定字符的情况
    if any(char in name for char in ['-', '/', '(', ')']):
        print(f'Skipping index {i} due to character filtering: {name}')
        hospital_link.append('Filtered')
        yiyuan.append(name)
        continue
    
    # 构建和发送请求
    url = f'{base_url}{name}'
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        p_tag = soup.find("p", class_="result")
        
        # 检查p_tag是否存在
        if p_tag and p_tag.find("b"):
            result = p_tag.find("b").text
            if result != '0':
                link = soup.find('div', class_='yy-cont').find('a')
                hospital_link.append(link['href'] if link else 'NA')
            else:
                hospital_link.append('NA')
        else:
            hospital_link.append('NA')
    else:
        hospital_link.append('Failed to fetch')
    
    yiyuan.append(name)

# 创建 DataFrame 并保存到Excel
new_df = pd.DataFrame({'Hospital': yiyuan, 'URL': hospital_link})
#new_df.to_excel('99health-new.xlsx', index=False)


In [8]:
new_df

,Hospital,URL
0,伊春市带岭区人民医院,NA
1,大庆市中西医结合医院,NA
2,大同区林源医院,NA
3,杜尔伯特蒙古族自治县中医医院,NA
4,杜尔伯特蒙古族自治县人民医院,NA
5,杜蒙县妇幼保健医院,NA
6,杜尔伯特蒙古族自治县妇幼保健计划生育服务中心,NA
7,大庆市萨尔图区陈立军内科诊所,NA
8,大庆油田四厂医院,NA
9,大庆油田总医院集团南区医院,NA


In [9]:
yiyuan

['伊春市带岭区人民医院',
 '大庆市中西医结合医院',
 '大同区林源医院',
 '杜尔伯特蒙古族自治县中医医院',
 '杜尔伯特蒙古族自治县人民医院',
 '杜蒙县妇幼保健医院',
 '杜尔伯特蒙古族自治县妇幼保健计划生育服务中心',
 '大庆市萨尔图区陈立军内科诊所',
 '大庆油田四厂医院',
 '大庆油田总医院集团南区医院',
 '大庆油田总医院集团铁人医院',
 '大庆创伤医院',
 '大庆市红岗区人民医院',
 '大庆市红岗区三九综合门诊',
 '大庆市红岗区欣成康养医院',
 '大庆市第二医院',
 '大庆市红岗区红杏综合门诊部',
 '齐齐哈尔爱民医院',
 '林甸县人民医院',
 '林甸县慈爱医院']

In [ ]:
base_url = 'https://www.dayi.org.cn/search?keyword='
yiyuan = []
hospital_link = []

for i in range(len(df["医院名称"])): 
    if any(char in df["医院名称"][i] for char in ['-', '/', '(', ')']):
        print(i)
        continue
    url = f'{base_url}{df["医院名称"][i]}'
    response = requests.get(url, headers=headers)
    content = response.text
    soup = BeautifulSoup(content, 'html.parser')
    p_tag = soup.find("p", class_="result")
    b_tag = find_b_tag(url, headers, p_tag)
    result = b_tag.get_text()
    if result !='0':
        link = soup.find('div', class_='yy-cont').find('a')
        if link:
            hospital_link.append(link['href'])
        else:
            hospital_link.append('NA')
    else:
        hospital_link.append('NA')
    yiyuan.append(df["医院名称"][i])
new_df = pd.DataFrame()
new_df['hospital'] = yiyuan
new_df['url'] = hospital_link
#new_df.to_excel('99health-new.xlsx',index=False)

In [ ]:
new_df.to_excel(r'C:\Users\cxy\OneDrive\桌面\回国\实习\7-29\院长查询测试.xlsx',index=False)
df = pd.read_excel('院长查询测试.xlsx')
df

In [ ]:
df_drop.reset_index(drop=True, inplace=True)

In [9]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

# 假设 df_drop 已经包含了你想处理的数据，且已经删除了包含 NaN 的行
# 例如：df_drop = df_cleaned （上面清理后的 DataFrame）
df_drop=df
# 初始化列表以保存数据
hospital_name = []
labels = []
address = []
keshi = []
yiyuan = []
yz_div=[]
# 可能需要调整开始的索引为之前出错的第一个索引，比如 64
start_index = 0  # 调整为你之前错误后的第一个索引
end_index = len(df_drop['hospital'])  # 继续到数据集的结束

for i in np.arange(start_index, end_index):
    url = df_drop['url'][i]
    response = requests.get(url, headers=headers)
    content = response.text
    soup = BeautifulSoup(content, 'html.parser')
    div_class = soup.find("div", class_ = "wrap_title")
    
    if div_class:
        # 医院名称
        #hospital_name.append(div_class.find("b").get_text())
        div_tag=soup.find("div",class_="flex")
        if div_tag and div_tag.get_text()=="医院院长":
            yz_div.append(div_tag.find_next_sibling('span').get_text())
        else:
            yz_div.append('NA')
        # 标签
        #labels.append([j.get_text() for j in div_class.find_all('span')])
        # 地址
        #i_tag = soup.find('i', class_='wrap_yydz')
        #if i_tag and i_tag.find_parent('em'):
        #    address.append(i_tag.find_parent('em').find_next_sibling('span').get_text())
        #else:
        #    address.append('NA')
        # 科室信息
        #div_tag = soup.find("div", class_="left_title")
        #if div_tag and div_tag.get_text() == "医院科室":
         #   dept_texts = []
        #    next_div = div_tag.find_next_sibling('div')
        #    for dd in next_div.find_all('dd'):
         #       dept_texts.extend([a.get_text() for a in dd.find_all('a')])
            #keshi.append(", ".join(dept_texts))
        #else:
       #     keshi.append('NA')
    else:
       # hospital_name.append('NA')
        #labels.append('NA')
        #address.append('NA')
        #keshi.append('NA')
        yz_div.append('NA')
    
    # 每次循环，都添加当前医院的名称
    yiyuan.append(df_drop['hospital'][i])

# 创建 DataFrame
new_df = pd.DataFrame({
    'hospital': yiyuan,
    'yz':yz_div ,  # 确保这些字段也添加到 DataFrame 中
    #'labels': labels,
    #'address': address,
    #'keshi': keshi
})

new_df.to_csv(r"C:\Users\cxy\OneDrive\桌面\回国\实习\7-29\小规模东北测试_更新.csv", index=False)


KeyError: 'url'

In [42]:
new_df.to_csv(r"C:\Users\cxy\OneDrive\桌面\回国\实习\7-8\99health-keshi.csv", index=False, encoding='utf-8-sig')